Up until now, we have been using Uniprot keywords for annotations. In order to get all transmembrane transporters of Sugar, we filter the proteins in the dataset by the keywords "Transmembrane" and "Sugar transport". 

This works most of the time, but sometimes there is a small number of proteins that fulfils these requirements, but does not actually transport the substrate. That could be the case for a "sugar transporter" that exchanges ATP and ADP, which is in turn used to promote Sugar transport. This means that the protein we are looking at promotes sugar transport, but is not a sugar transporter by itself. 

In order to prevent these rare cases, we could turn to the Gene Ontology. The advantage here is that the "molecular function" GO terms are much more specific. We would not have to filter for multiple keywords, and could instead just filter for "sugar transmembrane transporter activity". 

Tasks:

- How many substrates do we get with keywords/go in comparison?
- What can we say about proteins that either have a go term or a keyword? Are they outliers, or are they simply not annotated correctly in GO/KW?